<a href="https://colab.research.google.com/github/sharadv99/capstone-OpioidAddictionRisk/blob/master/Initial_Data_Prep_and_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
import pickle
plt.style.use('fivethirtyeight')
%matplotlib inline

In [5]:
#Clone data into temp folder - must be run once at the beginning of every session!
!git clone -l -s https://github.com/sharadv99/capstone-OpioidAddictionRisk.git capstone
os.chdir('capstone')

fatal: destination path 'capstone' already exists and is not an empty directory.


In [0]:
#Read in all survey data from 2015-2017
with zipfile.ZipFile("NSDUH-2017-DS0001-bndl-data-tsv.zip") as z:
    data_2017 = pd.read_csv(z.open("NSDUH_2017_Tab.tsv"), sep="\t", low_memory= False)
with zipfile.ZipFile("NSDUH-2016-DS0001-bndl-data-tsv.zip") as z:
    data_2016 = pd.read_csv(z.open("NSDUH_2016_Tab.tsv"), sep="\t", low_memory= False)
with zipfile.ZipFile("NSDUH-2015-DS0001-bndl-data-tsv.zip") as z:
    data_2015 = pd.read_csv(z.open("NSDUH_2015_Tab.tsv"), sep="\t", low_memory= False)

In [0]:
data_2017.to_pickle('2017.pickle')
data_2016.to_pickle('2016.pickle')
data_2015.to_pickle('2015.pickle')

In [0]:
del data_2017
del data_2016
del data_2015

In [0]:
data_2015 = pd.read_pickle('2015.pickle')
data_2016 = pd.read_pickle('2016.pickle')
data_2017 = pd.read_pickle('2017.pickle')

In [10]:
#verify shapes
print(data_2017.shape)
print(data_2016.shape)
print(data_2015.shape) #cols that werent collected in 2016 and 2017 need to be removed

(56276, 2668)
(56897, 2668)
(57146, 2679)


In [0]:
#Based on PUFVariableCrossWalk2017 there are inconsistent column headers - change to match 2017
data_2015.rename(columns={"COLLENRST": 'COLLENRLST', 'COLLENRFT': 'COLLENRLFT'}, inplace=True)

In [12]:
#columns in 2016 that arent in 2017
data_2016.columns.difference(data_2017.columns)

Index(['PNROTNMYR3', 'SEDOTNMYR3', 'STMOTNMYR3', 'TRQOTNMYR3'], dtype='object')

In [13]:
#columns in 2017 that arent in 2016
data_2017.columns.difference(data_2016.columns)

Index(['EDUSCKCOM', 'EDUSCKEST', 'EDUSKPCOM', 'EDUSKPEST'], dtype='object')

In [0]:
#Begin removing inconsistent columns
data_2016.drop(['PNROTNMYR3', 'SEDOTNMYR3', 'STMOTNMYR3', 'TRQOTNMYR3'], axis = 1, inplace= True)
data_2017.drop(['EDUSCKCOM', 'EDUSCKEST', 'EDUSKPCOM', 'EDUSKPEST'], axis = 1, inplace=True)

In [0]:
#merge 2016 and 2017 data
data = pd.concat([data_2017, data_2016], ignore_index= True)
del data_2016
del data_2017

In [0]:
#columns in 2016/2017 that arent in 2015
missing_cols_2015 = data.columns.difference(data_2015.columns)
#columns in 2015 that arent in 2016/2017
extra_cols_2015 = data_2015.columns.difference(data.columns)

In [0]:
#Run only if we want to include 2015 data
data.drop(list(missing_cols_2015), axis = 1, inplace = True)
data_2015.drop(list(extra_cols_2015), axis = 1, inplace = True)

In [18]:
print(data.shape)
print(data_2015.shape)

(113173, 2631)
(57146, 2631)


In [19]:
#merge 2015 and 2016/2017 data
data = pd.concat([data, data_2015], ignore_index= True)
del data_2015
print(data.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(170319, 2631)


In [20]:
data.groupby(['PNRNMREC'])['QUESTID2'].count()

PNRNMREC
1       2531
2       6106
3       9355
8         46
9        122
11         1
83      1181
91    150299
98       678
Name: QUESTID2, dtype: int64

In [21]:
data.groupby(['PNRANYREC'])['QUESTID2'].count()

PNRANYREC
1     53358
2     38889
9       476
83      913
91    76108
98      575
Name: QUESTID2, dtype: int64

In [23]:
dfUsePainMeds = data[data.PNRANYREC==1]
dfUsePainMeds

,AALTMDE,ABODALC,ABODCOC,ABODHER,ABODMRJ,ABPYILANAL,ABPYILLALC,ABUPOSHAL,ABUPOSINH,ABUPOSMTH,ABUPOSPNR,ABUPOSSED,ABUPOSSTM,ABUPOSTRQ,ABUSEALC,ABUSECOC,ABUSEHER,ABUSEMRJ,ABUSEPYHAL,ABUSEPYIEM,ABUSEPYILL,ABUSEPYINH,ABUSEPYMTH,ABUSEPYPNR,ABUSEPYPSY,ABUSEPYSED,ABUSEPYSTM,ABUSEPYTRQ,ACOUNMDE,ACTD2001,ACTD7590,ACTD9001,ACTDEVER,ACTDPRIV,ACTDVIET,ADCOUNS,ADDPDISC,ADDPLSIN,ADDPPROB,ADDPR2WK,...,YURSFRND,YURSIDN2,YURSIDYR,YURSOTPP,YURSSCHL,YURSSOR,YURSSUIC,YUSCEMYR,YUSCPGYR,YUSWANGR,YUSWBKRU,YUSWDEPR,YUSWEATP,YUSWFEAR,YUSWFITE,YUSWFMLY,YUSWFRND,YUSWOTPP,YUSWSCHL,YUSWSCYR,YUSWSOR,YUSWSUIC,YUTPANGR,YUTPBKRU,YUTPDEPR,YUTPEATP,YUTPFEAR,YUTPFITE,YUTPFMLY,YUTPFRND,YUTPOTPP,YUTPSCHL,YUTPSOR,YUTPSTN2,YUTPSTYR,YUTPSUIC,ZALEPDAPYU,ZOHYANYYR2,ZOLPPDAPYU,ZOLPPDPYMU
19,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,2,99,99,99,99,2,2,99,99,99,99,99,99,99,99,99,99,2,99,99,99,99,99,99,99,99,99,99,99,99,99,999,2,99,0,0,0,0
20,0.0,1,0,0,0,0,1,93.0,91.0,91.0,91.0,91.0,91.0,91.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,99,99,99,99,99,99,99,1,1,1,1,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
21,NaN,0,0,0,0,0,0,91.0,93.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
23,NaN,0,0,0,0,0,0,91.0,91.0,91.0,93.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
33,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
34,NaN,0,0,0,0,0,0,91.0,91.0,91.0,0.0,91.0,0.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
36,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2,2,2,1,1,2,99,99,99,99,2,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
46,NaN,0,0,0,0,0,0,0.0,93.0,91.0,0.0,91.0,91.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
50,NaN,0,0,0,0,0,0,91.0,93.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,999,99,99,0,0,0,0
52,NaN,0,0,0,0,0,0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,99,99,99,99,99,99,99,99,99,99,99,...,99,999,2,99,99,99,99,2,2,99,99,99,99,99,99,99,99,99,99,2,99,99,99,99,99,99,99,99,99,99,99,99,99,999,2,99,0,0,0,0


In [0]:
#Set MISUSE variable
misuseCodes = [1, 2, 8]
dfUsePainMeds['MISUSE'] = dfUsePainMeds.apply(lambda row:
                                              1 if row.PNRNMREC in misuseCodes else 0,
                                              axis=1)

dfUsePainMeds

In [0]:
#Determine classification imbalance
print('COUNT of MISUSE')
print(dfUsePainMeds.groupby(['MISUSE'])['MISUSE'].count())
class_imbal = dfUsePainMeds.groupby(['MISUSE'])['MISUSE'].count()
print('\nPERCENT MISUSE')
print(sum(dfUsePainMeds.MISUSE) / len(dfUsePainMeds))

In [0]:
list_of_vals = ["{:0.1f}%".format(100 * class_imbal[0]/len(dfUsePainMeds)),
                "{:0.1f}%".format(100 * class_imbal[1]/len(dfUsePainMeds))]
fig, ax = plt.subplots(1,1, figsize = (6,4))
ax.bar([0,1], list(dict(class_imbal).values()))
my_xticks = ['Did Not Misuse','Misuse']
ax.set_xticks([0,1])
ax.set_xticklabels(my_xticks)
ax.set_title("Patient Count")
ax.set_xlabel('Classes')
ax.set_ylabel('Count')
rects = ax.patches
for rect, label in zip(rects, list_of_vals):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
            ha='center', va='bottom', fontsize = 12, color = "r")




In [0]:
dep_var_counts = data["PNRNMINIT"].astype("str").value_counts()

In [0]:
dep_var_counts.plot(kind='bar', figsize = (8,6), title= "Frequency of Misuse by code")

In [0]:
for i,j in list(dep_var_counts.to_dict().items()):
    print("{} makes up {}% of the total".format(i,100*j/sum(list(dep_var_counts.to_dict().values()))))

In [0]:
unique_vals_per_col = data.nunique()

In [0]:
#x = Unique vals in a column   y = Number of cols that share that unique value
freq_counts = pd.Series(list(unique_vals_per_col)).value_counts().to_dict()
freq_counts = sorted(freq_counts.items())
freq_counts

In [0]:
x,y = zip(*freq_counts)
plt.bar(list(x), list(y))
plt.xlim([0,70])
plt.title("histogram of unique column value counts")
plt.xlabel("Unique Values in Column")
plt.ylabel("Number of Columns")